In [1]:
from langchain.vectorstores import FAISS
import os
from openai import AzureOpenAI
import openai
from dotenv import load_dotenv
import os
load_dotenv()

True

In [25]:
# COHERE_API_KEY = 'c1ui1Fu9cHUtADtZBWkSkYxkU5gPI3KdG3bzkQSa'
# co = cohere.Client(COHERE_API_KEY)
from langchain_openai.chat_models import AzureChatOpenAI
import os
from dotenv import load_dotenv
load_dotenv()
# co = AzureChatOpenAI(
#     openai_api_version=os .environ.get("AZURE_OPENAI_VERSION", "2024-07-18"),
#     azure_deployment=os.environ.get("AZURE_OPENAI_DEPLOYMENT", "gpt-4o-mini"),
#     azure_endpoint=os.environ.get("AZURE_OPENAI_ENDPOINT", "https://gptmini4o.openai.azure.com/openai/deployments/gpt-4o-mini/chat/completions?api-version=2023-03-15-preview"),
#     api_key=os.environ.get("AZURE_OPENAI_KEY", "c2105be0c2744742980b57320b87e813")
# )

True

In [3]:
import pandas as pd
with open('haystack.txt', 'r', encoding='utf-8') as file:
    text = file.read()

In [4]:
list_text = text.split('\n')

In [5]:
len(list_text)

599300

In [6]:
from langchain_text_splitters import CharacterTextSplitter, RecursiveCharacterTextSplitter

text_splitter = CharacterTextSplitter(
    separator="\n\n",
    chunk_size=1000,
    chunk_overlap=200,
    length_function=len,
    is_separator_regex=False,
)

rec_text_splitter = RecursiveCharacterTextSplitter(
    # Set a really small chunk size, just to show.
    chunk_size=1000,
    chunk_overlap=200,
    length_function=len,
    is_separator_regex=False,
    separators=["\n\n", "\n", "\n\n\n", "."],
)

In [7]:
texts_rec = rec_text_splitter.create_documents([text])

In [8]:
print(texts_rec[2256])

page_content='hooded head moved only when he cocked an ear at an 
interesting sound. 

Through the day’s gathering crowds the strange pair 
came, arriving at last on the steps which led up like terraced 
hectares to the escarpment which was Alia’s Temple, a 
fitting companion to Paul’s Keep. Up the steps The Preacher 
went until he and his young guide came to the third landing, 
where pilgrims of the Hajj awaited the morning opening of 
those gigantic doors above them. They were doors large 
enough to have admitted an entire cathedral from one of 
the ancient religions. Passing through them was said to 
reduce a pilgrim’s soul to motedom, sufficiently small that it 
could pass through the eye of a needle and enter heaven.'


In [9]:
len(texts_rec)

26909

In [13]:
from langchain_community.embeddings import OpenAIEmbeddings
openai = OpenAIEmbeddings(openai_api_key=os.environ.get("OPENAI_API_KEY"))

/var/folders/b0/y0q5l7lx61g3k6f4zvy9__4h0000gn/T/ipykernel_39925/4217019477.py:2: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  openai = OpenAIEmbeddings(openai_api_key=os.environ.get("OPENAI_API_KEY"))


In [14]:
from langchain_community.vectorstores import Chroma
vectorstore = Chroma.from_documents(documents=texts_rec, embedding=openai)

In [18]:
retriever = vectorstore.as_retriever()

In [19]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

In [27]:
model = ChatOpenAI(model="gpt-4o-mini", api_key=os.environ.get("OPENAI_API_KEY"))

In [28]:
template = """Answer the question based only on the following context:

{context}

Here is how the answer should look like:- Ryoshi, based in Neo Tokyo, Japan, is a private quantum computing firm founded in 2031, currently valued at $8.7 billion with 1,200 employees focused on quantum cryptography.

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

In [37]:
chain = (
    {"context": retriever , "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

'The text that is repeated and not related to the story includes:\n\n1. "On Anteac’s lap lay a small square of inky black about ten millimeters on a side and no more than three millimeters thick. She wrote upon this square with a glittering needle— one word upon another, all of them absorbed into the Square. The completed message would be impressed upon the nerve receptors of an acolyte-messenger’s eyes, latent there until they could be replayed at the Chapter House."\n\n2. "Hwi Noree posed such a dilemma!"\n\n3. "Anteac knew the accounts of Bene Gesserit teachers sent to instruct Hwi on Ix. But those accounts left out more than they told. They raised greater questions."\n\n4. "What adventures have you experienced, child?"\n\n5. "What were the hardships of your youth?"\n\n6. "Anteac sniffed and glanced down at the waiting square of black. Such thoughts reminded her of the Fremen belief that the land of your birth made you what you were."\n\n7. "“Are there strange animals on your planet

In [38]:
from langchain.retrievers.multi_vector import MultiVectorRetriever
from langchain.storage import InMemoryByteStore
from langchain_community.vectorstores import Chroma
from langchain_community.document_loaders import TextLoader
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [10]:
# hf_token = os.getenv("HF_TOKEN")
# from langchain.embeddings import HuggingFaceInferenceAPIEmbeddings
# embeddings = HuggingFaceInferenceAPIEmbeddings(
#         api_key=hf_token, model_name="BAAI/bge-base-en-v1.5"
#     )

In [39]:
loaders = [
    TextLoader("haystack.txt"),
]
docs = []
for loader in loaders:
    docs.extend(loader.load())
text_splitter = RecursiveCharacterTextSplitter(chunk_size=10000)
docs = text_splitter.split_documents(docs)

In [42]:
# The vectorstore to use to index the child chunks
vectorstore = Chroma(
    collection_name="full_documents", embedding_function=openai
)
# The storage layer for the parent documents
store = InMemoryByteStore()
id_key = "doc_id"
# The retriever (empty to start)
retriever = MultiVectorRetriever(
    vectorstore=vectorstore,
    byte_store=store,
    id_key=id_key,
)
import uuid

doc_ids = [str(uuid.uuid4()) for _ in docs]

/var/folders/b0/y0q5l7lx61g3k6f4zvy9__4h0000gn/T/ipykernel_39925/3023248933.py:2: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vectorstore = Chroma(


In [43]:
# The splitter to use to create smaller chunks
child_text_splitter = RecursiveCharacterTextSplitter(chunk_size=400)

In [44]:
sub_docs = []
for i, doc in enumerate(docs):
    _id = doc_ids[i]
    _sub_docs = child_text_splitter.split_documents([doc])
    for _doc in _sub_docs:
        _doc.metadata[id_key] = _id
    sub_docs.extend(_sub_docs)

In [46]:
retriever.vectorstore.add_documents(sub_docs)
retriever.docstore.mset(list(zip(doc_ids, docs)))

ValueError: Batch size 86718 exceeds maximum batch size 41666

In [ ]:
from langchain.retrievers.multi_vector import SearchType

retriever.search_type = SearchType.mmr

len(retriever.invoke("Give me all the cancer terms that ")[0].page_content)